**Это функция для сбора данных из Google Sheets, которая принимает три списки (данные с каждого листа в Google Sheets), определяет долги по платежам на указанную дату и сохраняет отчет в файл `report.txt`**

In [94]:
import gspread # gspread используем для работы с Google Sheets API
                # С его помощью можно создавать, редактировать и читать данные из таблиц.
from oauth2client.service_account import ServiceAccountCredentials # Чтобы получить доступ к Google API, нужно использовать учетные данные службы.
                                                                    # Для этого понадобится модуль oauth2client.service_account, который содержит класс ServiceAccountCredentials.
                                                                    # Он и позволяет создавать объекты учетных данных службы для аутентификации.
# файл creds.json, нужно будет передать в ServiceAccountCredentials.from_json_keyfile_name(), а затем передать все в gspread.authorize().
# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
        'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds) #Теперь можем работать с Googdle Sheet прямо из Googe Colab.


#Получаем список словарей-строк из Лист1, Лист2, Лист 3 (Google Sheets)
sheet1 = client.open("Installments").worksheet("Лист1")
sheet1_data = sheet1.get_all_records() #[{key1:value1},{key2:value2},{key3:value3}]

sheet2 = client.open("Installments").worksheet("Лист2")
sheet2_data = sheet2.get_all_records()

sheet3 = client.open("Installments").worksheet("Лист3")
sheet3_data = sheet3.get_all_records()

# API ПОДКЛЮЧЕН И ДАННЫЕ ЗАКГРУЖЕННЫ
# ТЕПЕРЬ САМА ФУНКЦИЯ
def generate_report(sheet1, sheet2, sheet3):

  #отфильтруем "Лист 1", оставив только записи с меткой "Y" в поле "installment"
  sheet1_data_instmnt = []#создаем пустой список для добавления данных с меткой 'Y'
  for row1 in sheet1_data:
    if row1['installment'] =='Y':
      sheet1_data_instmnt.append({**row1})

  # РАСЧЕТ КОЛИЧЕСТВА ПРОШЕДШИХ ДНЕЙ С ДАТЫ ОЖИДАЕМОГО ПЛАТЕЖА ДО КОНТРОЛЬНОЙ ДАТЫ d1 = 01.03.2023
  import math
  from datetime import datetime
  date1_str = '01.03.2024'#-Контрольная текущая дата
  date1 = datetime.strptime(date1_str, '%d.%m.%Y')#создем дату как instance of datetime class

  with open('debt_report.txt', 'w') as debt_list:
    for row1 in sheet1_data_instmnt:
      for row2 in sheet2_data:
        if row1['id'] == row2['id']:# проверка реляции строки по id
          #определяем есть ли долг
          #рассчитываем кол-во дней с даты последнего платежа
          date2_str = row2['last_payment_date'] #expected_payment_date
          date2 = datetime.strptime(date2_str, '%d.%m.%Y')#создем дату как instance of datetime class
          days = (date1 - date2).days #применяем метод класса для расчета количества дней
         
          days_ratio = floor(days / 180) #если  >= 1 значит прошло более 183 дн с даты последнего платежа и есть долг

          if days_ratio >= 1: # - есть долг!
            for row in sheet3_data: #ищем запись по id в таблице платежей
              if row['id'] == row1['id']: # проверка реляции строки по id
                debt = (row['left_to_pay'] - ((days_ratio * row['one-time_payment'] ) )) # рассчитываем сумму долга
                if debt < 0:
                  debt_str = f"Клиент {row1['name']} - долг {row['left_to_pay']} рублей"
                  debt_list.write(debt_str + '\n')

                else:
                  debt_str = f"Клиент {row1['student_name']} - долг {(days_ratio * row['one-time_payment'])} рублей"
                  debt_list.write(debt_str + '\n')

  #ЭТО КОСТЫЛЬ - Удаляем последнюю пустую строку в файле
  with open('debt_report.txt', "r+") as f:
    # Переместите курсор в конец файла
    f.seek(0, 2)

    # Удалите последнюю строку путем усечения файла до размера курсора
    f.truncate(f.tell() - 1)

res = generate_report(sheet1, sheet2, sheet3)